In [5]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from sklearn.model_selection import train_test_split

In [8]:
# Load data from pickle file
data = pickle.load(open("D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/data_face_features.pickle", mode="rb"))

In [10]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.02926854  0.2144657  -0.03790816 ... -0.06640885  0.01369474
    0.06734287]]

 [[-0.0145682   0.00786272  0.08517676 ...  0.11353577  0.04404245
    0.12897009]]

 [[ 0.05722998 -0.0117318  -0.03553959 ...  0.15690751 -0.08634207
    0.00904021]]

 ...

 [[ 0.04576913  0.07487009 -0.02625914 ...  0.04409239  0.10175743
    0.07864672]]

 [[ 0.01342397  0.21639296  0.14320451 ... -0.04018039  0.19147922
    0.07989911]]

 [[ 0.03090836  0.19090441  0.10911512 ... -0.07382147  0.16626082
    0.07312067]]]
['151901205_BuiKhanhBinh' '151901205_BuiKhanhBinh'
 '151901205_BuiKhanhBinh' ... '1721031731_NguyenDongHung'
 '1721031731_NguyenDongHung' '1721031731_NguyenDongHung']


In [11]:
x.shape

(4337, 1, 128)

In [14]:
y.shape

(4337,)

In [15]:
x = x.reshape(-1, 128)

In [16]:
x.shape

(4337, 128)

In [20]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [21]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3469, 128), (868, 128), (3469,), (868,))

In [22]:
# 2. Train machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [23]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [24]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [25]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.70
F1 Score Train = 0.50
F1 Score Test = 0.48


In [29]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [31]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.84
F1 Score Train = 0.82
F1 Score Test = 0.74


In [32]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [34]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.66


In [36]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [37]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [38]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.84
F1 Score Train = 0.97
F1 Score Test = 0.74


In [39]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [41]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   3.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   3.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   3.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   3.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   3.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   3.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   3.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   3.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   3.1s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [42]:
model_best_estimator = model_grid.best_estimator_

In [45]:
model_grid.best_score_

0.8486572879189808

In [48]:
# 4. Save Model
pickle.dump(model_best_estimator, open("D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/ml_face_person_identity.pkl", mode='wb'))